### create differing weekly dataframes for each position and each weekly with data for all players

### set up to get data for each week, and for each position, reasses data types,make sure all data is still accurate
### after all dataframe is set and ordered, transport to a storage system

In [1]:
#import libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import threading

In [2]:
#start driver for Chrome
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [3]:
#Driver start on fantasy pro site
driver.get("https://www.fantasypros.com/nfl/stats/qb.php?")
driver.maximize_window()

In [ ]:
#Holds query value of roles
NFL_Roles=["qb","rb","wr","te","k","dst","dl","lb","db"]
nested_dictionary_frames = {}

#start of loop to collect data for all roles in each week seperately
for role in NFL_Roles: 
    # Retrieve headers of the website dataframe
    header = driver.find_elements(By.XPATH, "//tr[@class='tablesorter-header']//th")
    NFL_Labels = [label.text for label in header]
    Label_Size = len(NFL_Labels)
    df_weeks = []
    nested_dictionary_frames[role] = {}
    for week in range(1, 12):
        gen_data = []
        # Retrieval of website 
        driver.get(f"https://www.fantasypros.com/nfl/stats/{role}.php?week={week}&range=week&scoring=PPR")
        # Obtain all player data presented
        table_data = driver.find_elements(By.XPATH, "//table[@id='data']//tbody//td")
        #collect all data 
        for info in table_data:
            gen_data.append(info.text)
        row_data = [gen_data[x:x + Label_Size] for x in range(0, len(gen_data), Label_Size)]
        df_week = pd.DataFrame(row_data, columns=NFL_Labels)
        df_week["week"] = week
        df_weeks.append(df_week)
        
    nested_dictionary_frames[role] = df_weeks

In [ ]:
def scrape_nfl_data(start_week, end_week):
    #
    NFL_Roles=["qb","rb","wr","te","k","dst","dl","lb","db"]
    nested_dictionary_frames = {}
    for role in NFL_Roles:
        driver.get(f"https://www.fantasypros.com/nfl/stats/{role}.php")
        header = driver.find_elements(By.XPATH, "//tr[@class='tablesorter-header']//th")
        NFL_Labels = [label.text for label in header]
        Label_Size = len(NFL_Labels)
        sleep(3)  
        df_weeks = []
        nested_dictionary_frames[role] = {}
        for week in range(start_week, end_week + 1):
            gen_data = []
            driver.get(f"https://www.fantasypros.com/nfl/stats/{role}.php?week={week}&range=week&scoring=PPR")
            table_data = driver.find_elements(By.XPATH, "//table[@id='data']//tbody//td")
            for info in table_data:
                gen_data.append(info.text)
            row_data = [gen_data[x:x + Label_Size] for x in range(0, len(gen_data), Label_Size)]
            df_week = pd.DataFrame(row_data, columns=NFL_Labels)
            #df_week["TEAM"] = df_week["PLAYER"].split("(")[0]
            df_week["WEEK"] = week
            df_weeks.append(df_week)
        nested_dictionary_frames[role] = df_weeks
        for num_col in range(start_week - 1, end_week):
            nested_dictionary_frames[role][num_col].iloc[:, 2:] = nested_dictionary_frames[role][num_col].iloc[:, 2:].apply(pd.to_numeric, errors="ignore", downcast="float")
    driver.quit()        
    return nested_dictionary_frames 

In [ ]:
result=scrape_nfl_data(1,11)

In [ ]:
result["db"][0]

In [1]:
for role, frames_list in result.items():
        for week, df_week in enumerate(frames_list, 1):
            df_week.to_csv(f"{role}_week_{week}.csv", index=False)

NameError: name 'result' is not defined

In [ ]:
for role in NFL_Roles:
    for given in range(0,11):
        nested_dictionary_frames[role][given]=nested_dictionary_frames[role][given].iloc[:,2:].apply(pd.to_numeric,errors="ignore",downcast="float")

In [50]:
nested_dictionary_frames["qb"][3]